In [25]:
from torchinfo import summary
from transformers import PegasusTokenizer, PegasusTokenizerFast, PegasusModel, PegasusForConditionalGeneration, PegasusConfig

pegasus_model = PegasusForConditionalGeneration.from_pretrained("sshleifer/distill-pegasus-cnn-16-4")
summary(pegasus_model)

Layer (type:depth-idx)                                       Param #
├─PegasusModel: 1-1                                          --
|    └─Embedding: 2-1                                        98,409,472
|    └─PegasusEncoder: 2-2                                   --
|    |    └─Embedding: 3-1                                   (recursive)
|    |    └─PegasusSinusoidalPositionalEmbedding: 3-2        (1,048,576)
|    |    └─ModuleList: 3-3                                  201,539,584
|    |    └─LayerNorm: 3-4                                   2,048
|    └─PegasusDecoder: 2-3                                   --
|    |    └─Embedding: 3-5                                   (recursive)
|    |    └─PegasusSinusoidalPositionalEmbedding: 3-6        (1,048,576)
|    |    └─ModuleList: 3-7                                  67,186,688
|    |    └─LayerNorm: 3-8                                   2,048
├─Linear: 1-2                                                98,409,472
Total params: 467,646,46

In [33]:
PegasusTokenizer.from_pretrained("sshleifer/distill-pegasus-cnn-16-4").vocab_size

96103

In [32]:
configuration = PegasusConfig(d_model=768, vocab_size=tokenizer.vocab_size, 
                              encoder_layers=8, decoder_layers=8, 
                              encoder_attention_heads=12, decoder_attention_heads=12,
                              decoder_ffn_dim=3072, encoder_ffn_dim=3072)
model = PegasusForConditionalGeneration(configuration)
configuration = model.config
summary(model)

Layer (type:depth-idx)                                       Param #
├─PegasusModel: 1-1                                          --
|    └─Embedding: 2-1                                        73,807,104
|    └─PegasusEncoder: 2-2                                   --
|    |    └─Embedding: 3-1                                   (recursive)
|    |    └─PegasusSinusoidalPositionalEmbedding: 3-2        (786,432)
|    |    └─ModuleList: 3-3                                  56,702,976
|    |    └─LayerNorm: 3-4                                   1,536
|    └─PegasusDecoder: 2-3                                   --
|    |    └─Embedding: 3-5                                   (recursive)
|    |    └─PegasusSinusoidalPositionalEmbedding: 3-6        (786,432)
|    |    └─ModuleList: 3-7                                  75,614,208
|    |    └─LayerNorm: 3-8                                   1,536
├─Linear: 1-2                                                73,807,104
Total params: 281,507,328
Tra

In [1]:

%reload_ext autoreload

In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
!nvidia-smi

Fri Apr 30 19:00:33 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  On   | 00000000:2D:00.0 Off |                  N/A |
|  0%   58C    P2   243W / 300W |   5481MiB / 11016MiB |     96%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [ ]:
from dataset import CnnDmDataset
from torch.utils.data import DataLoader

dataset_train = CnnDmDataset('train')
dataset_val = CnnDmDataset('validation')

print()

BATCH_SIZE = 3

dataloader_train = DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True, num_workers=16, collate_fn=CnnDmDataset.collate_fn)
dataloader_val = DataLoader(dataset_val, batch_size=2*BATCH_SIZE, shuffle=True, num_workers=8, collate_fn=CnnDmDataset.collate_fn)

In [ ]:
%timeit [dataset_train[i] for i in range(100)]

17.7 ms ± 786 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
%timeit [dataset_train[i]['article'] for i in range(100)]

/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:38: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/home/kdchernyshev/sum/dataset.py:34: RuntimeWarning: invalid value encountered in true_divide
  node_features = node_features / np.clip(node_features.sum(1), a_min=1, a_max=None)[:,None]
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:38: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


422 ms ± 101 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%timeit [dataset_train[i] for i in range(100)]

604 ms ± 35.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
from datasets import load_metric

metric_rouge = load_metric('rouge')
metric_meteor = load_metric('meteor')
# metric_rouge.add_batch(predictions=i['article'], references=i['summary'])
# metric_rouge.compute()

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/kdchernyshev/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
from models.gat import GAT, GraphEncoder

gat_model = GAT(num_of_layers=2, num_heads_per_layer=[8, 8], num_features_per_layer=[128, 64, 32])
gat_enc_model = GraphEncoder(num_of_layers=2, num_heads_per_layer=[8, 8], num_features_per_layer=[128, 64, 32])

data = dataset_train[0]
data1 = dataset_train[1]

print('data', data['node_features'][None].shape, data['topology'][None].shape)

out_nodes_features, edge_index = gat_model(data['node_features'], data['topology'])
print(out_nodes_features.shape)

print('--')


out_nodes_features_batch, edge_index_batch = gat_enc_model([data['node_features'], data1['node_features']], [data['topology'], data1['topology']])
print([i.shape for i in out_nodes_features_batch])

data torch.Size([1, 379, 128]) torch.Size([1, 2, 648])
torch.Size([379, 32])
--
[torch.Size([379, 32]), torch.Size([211, 32])]


In [ ]:
from models import get_model

tokenizer, model = get_model()
model = model.cpu()

from torchinfo import summary
summary(model, depth=3)

Layer (type:depth-idx)                                  Param #
├─Embedding: 1-1                                        (98,409,472)
├─PegasusEncoder: 1-2                                   --
|    └─Embedding: 2-1                                   (recursive)
|    └─PegasusSinusoidalPositionalEmbedding: 2-2        (1,048,576)
|    └─ModuleList: 2-3                                  --
|    |    └─PegasusEncoderLayer: 3-1                    (12,596,224)
|    |    └─PegasusEncoderLayer: 3-2                    (12,596,224)
|    |    └─PegasusEncoderLayer: 3-3                    (12,596,224)
|    |    └─PegasusEncoderLayer: 3-4                    (12,596,224)
|    └─LayerNorm: 2-4                                   (2,048)
├─GraphEncoder: 1-3                                     --
|    └─GAT: 2-5                                         --
|    |    └─ModuleList: 3-5                             (68,187,136)
├─PegasusSuperDecoder: 1-4                              --
|    └─Embedding: 2-6      

In [ ]:
model.__dict__

{'training': True,
 '_parameters': OrderedDict(),
 '_buffers': OrderedDict(),
 '_non_persistent_buffers_set': set(),
 '_backward_hooks': OrderedDict(),
 '_is_full_backward_hook': None,
 '_forward_hooks': OrderedDict(),
 '_forward_pre_hooks': OrderedDict(),
 '_state_dict_hooks': OrderedDict(),
 '_load_state_dict_pre_hooks': OrderedDict(),
 '_modules': OrderedDict([('shared', Embedding(96103, 1024, padding_idx=0)),
              ('encoder',
               PegasusEncoder(
                 (embed_tokens): Embedding(96103, 1024, padding_idx=0)
                 (embed_positions): PegasusSinusoidalPositionalEmbedding(1024, 1024)
                 (layers): ModuleList(
                   (0): PegasusEncoderLayer(
                     (self_attn): PegasusAttention(
                       (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
                       (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
                       (q_proj): Linear(in_features=1024, 

In [ ]:
import torch
import torch.nn.functional as F

data = dataset_train[0]
data1 = dataset_train[1]
input_tokens = tokenizer([data['article'], data1['article']], truncation=True, padding='longest', return_tensors="pt")
target_tokens = tokenizer([data['summary'], data1['summary']], truncation=True, padding='longest', return_tensors="pt")

out = model(**input_tokens, decoder_input_ids=target_tokens.input_ids, input_nodes_embeddings=[data['node_features'], data1['node_features']], input_edges=[data['topology'], data1['topology']])
print(tokenizer.batch_decode(torch.argmax(F.softmax(out.logits, dim=-1), dim=-1)))
# print(tokenizer.batch_decode(model.predict(**input_tokens)))

encoder_hidden_states torch.Size([2, 1024, 1024])
encoder_attention_mask torch.Size([2, 1024])
gat_hidden_states torch.Size([2, 379, 1024])
gat_attention_mask torch.Size([2, 379])
['Syrian  I has to to the to the House to andThis\'t be" we to to" The has a letter to the House ofs House on Senate on  The has make out approval for Saturday issues: the,. The of to be how the weapons used by and just</s>, the the-S. president  </s></s>', " Bolt the in in     Theing:' winxxs and and in </s>-s the end, the and </s>'- for the's finalthxm and  </s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s>"]


In [ ]:
from openie import get_triplets
import torch

triplets = get_triplets(data['summary'])

nodes = set()
for t in triplets:
    nodes.update(t)
nodes = {n: i for i, n in enumerate(nodes)}

def _get_nodes_features(nodes, enc_dim=1024, device='cuda'):
    emb = torch.zeros(size=(len(nodes), enc_dim))

    texts = [text for text, i in nodes.items()]

    input_tokens = tokenizer(texts, truncation=True, padding='longest', return_tensors="pt")
    input_tokens = {k: v.to(device) for k, v in input_tokens.items()}

    with torch.no_grad():
        out = model.encoder(input_tokens['input_ids'], input_tokens['attention_mask'])

    for k, v in input_tokens.items(): del v

    # del out

    return out.last_hidden_state
        
_get_nodes_features(nodes)

BaseModelOutput(last_hidden_state=tensor([[[ 0.2346, -0.0819, -0.0281,  ..., -0.0959, -0.6350, -0.0228],
         [-0.0315, -0.0278,  0.1546,  ...,  0.0752,  0.0322,  0.0144],
         [ 0.0681, -0.0955,  0.0784,  ...,  0.1872, -0.0754,  0.1290],
         ...,
         [ 0.0288, -0.0996,  0.0833,  ...,  0.1636, -0.0811,  0.1207],
         [ 0.0259, -0.0962,  0.0886,  ...,  0.1305, -0.0849,  0.1317],
         [ 0.0228, -0.0887,  0.0967,  ...,  0.1045, -0.0515,  0.1379]],

        [[ 0.0454, -0.0306,  0.0409,  ...,  0.0661, -0.0218,  0.0676],
         [-0.0288, -0.0265,  0.1581,  ...,  0.1106,  0.0576,  0.0338],
         [ 0.0790, -0.0706,  0.0901,  ...,  0.2093, -0.0231,  0.1510],
         ...,
         [ 0.0506, -0.0821,  0.0948,  ...,  0.1686, -0.0789,  0.1438],
         [ 0.0237, -0.0705,  0.0938,  ...,  0.1311, -0.0561,  0.1128],
         [ 0.0231, -0.0655,  0.0949,  ...,  0.1354, -0.0537,  0.1188]],

        [[ 0.0043, -0.0469, -0.0045,  ...,  0.2118,  0.2374,  0.0231],
         [-

In [ ]:
model.encoder(**input_tokens)

BaseModelOutput(last_hidden_state=tensor([[[ 0.0544, -0.0105,  0.0110,  ...,  0.0767, -0.0407, -0.0432],
         [ 0.0256,  0.0374, -0.0972,  ...,  0.1123,  0.0054, -0.0293],
         [ 0.2315,  0.0837, -0.0035,  ..., -0.1696,  0.0024, -0.0686],
         ...,
         [-0.2165,  0.1029, -0.0683,  ..., -0.2700, -0.1096,  0.0844],
         [-0.0412,  0.0570,  0.0067,  ...,  0.1963, -0.1248,  0.0603],
         [-0.0488,  0.0173,  0.1425,  ...,  0.0509,  0.1813,  0.0235]]],
       device='cuda:0'), hidden_states=None, attentions=None)

In [ ]:
import torch
import torch.nn.functional as F
data = dataset_train[0]
input_tokens = tokenizer(data['article'], truncation=True, padding='longest', return_tensors="pt")
target_tokens = tokenizer(data['summary'], truncation=True, padding='longest', return_tensors="pt")

tokenizer.batch_decode(torch.argmax(F.softmax(model(**input_tokens, decoder_input_ids=target_tokens.input_ids).logits, dim=-1), dim=-1))

RuntimeError: ignored

In [ ]:
tokenizer.batch_decode(model.predict(**input_tokens))

['<pad> The has to be by the the the House House House is to be for for for first to have been by the House wants to hold on how will be by President President President Obama has to take, on issues</s>']

In [ ]:
data = dataset_train[0]
input_tokens = tokenizer(data['article'], truncation=True, padding='longest', return_tensors="pt")
target_tokens = tokenizer(data['summary'], truncation=True, padding='longest', return_tensors="pt")

tokenizer.batch_decode(model.generate(**input_tokens, do_sample=True, repetition_penalty=2.0, min_length=42, max_length=100, no_repeat_ngram_size=3, num_beams=4))

['<pad> President Obama has to be by the the the House House House is to hold on the the security and officials have been by the House wants to make for for for first to be of the the North to protect, on issues</s>']

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss
import torch.nn.functional as F
from tqdm.auto import tqdm

In [ ]:
from common import get_criterion_loss

In [ ]:
import neptune.new as neptune

run = neptune.init(project='k4black/diploma-sum', api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIzZDliYzEyZS1hOWVkLTQ1ZDQtOThlYS1jNDhhOTFjMGQ4ZjAifQ==')

https://app.neptune.ai/k4black/diploma-sum/e/SUM-20


In [ ]:
from transformers import AdamW, get_cosine_schedule_with_warmup

LR = 1e-4
EPOCHES = 1
DEVICE = 'cuda:1'
MIN_GEN = 42
# DEVICE = 'cpu'
model = pegasus_model
tokenizer = pegasus_tokenizer

model = model.to(DEVICE)
model = nn.DataParallel(model, device_ids=[1, 4, 5, 6]).to(DEVICE)
# torch.distributed.init_process_group(backend='nccl', init_method='env://')
# model = nn.parallel.DistributedDataParallel(model, device_ids=[1, 3, 4, 5, 6]).to(DEVICE)

scaler = torch.cuda.amp.GradScaler()  # mixed precision 

criterion = lambda x, y, m: get_criterion_loss(x, y, m)
optimizer = AdamW(model.parameters(), lr=LR)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=100, num_training_steps=80000)


run["parameters"] = {"learning_rate": LR,
                     "model": "pegasus",
                     "encoders": "16",
                     "decoders": "4",
                     "optimizer": type(optimizer).__name__}

for e in tqdm(range(EPOCHES), desc="epoch", leave=False):
    train_runnig_loss = 0
    train_runnig_num = 0
    for i, batch in tqdm(enumerate(dataloader_train), total=len(dataloader_train), desc="train", leave=False):
        optimizer.zero_grad()

        input_tokens = tokenizer(batch['article'], truncation=True, padding='longest', return_tensors="pt")
        target_tokens = tokenizer(batch['summary'], truncation=True, padding='longest', return_tensors="pt")

        input_tokens = {k: v.to(DEVICE) for k, v in input_tokens.items()}
        target_tokens = {k: v.to(DEVICE) for k, v in target_tokens.items()}

        with torch.cuda.amp.autocast():
            outputs = model(**input_tokens, decoder_input_ids=target_tokens['input_ids'])
            loss = criterion(outputs.logits, target_tokens['input_ids'], target_tokens['attention_mask'])

        train_runnig_loss += loss.item()
        train_runnig_num += len(batch['article'])

        run["train/loss"].log(loss.item() / len(batch['article']))

        if i % 1 == 0:
            print('---')
            print("  running loss:", train_runnig_loss / train_runnig_num)
            run["train/running-loss"].log(train_runnig_loss / train_runnig_num)

            # if i % 64 == 0:
            #     with torch.no_grad():
            #         out = tokenizer.batch_decode(torch.argmax(F.softmax(outputs.logits, dim=-1), dim=-1))
            #         print('-> ', out)
            #         # print('vs ', batch['summary'])
            #         print('vs ', tokenizer.batch_decode(target_tokens['input_ids']))
            #         print('gn ', tokenizer.batch_decode(model.module.generate(**input_tokens, min_length=MIN_GEN, do_sample=True, num_beams=3).to('cpu')) )

            
            print('---')

        scaler.scale(loss).backward()  #
        scaler.step(optimizer)  # 
        # loss.backward()
        # optimizer.step()

        scheduler.step()
        scaler.update()

        for k, v in input_tokens.items(): del v
        for k, v in target_tokens.items(): del v

        if i > 100:
            break
                
    val_runnig_loss = 0
    val_runnig_num = 0
    for i, batch in tqdm(enumerate(dataloader_val), total=len(dataloader_val), desc="val", leave=False):
        with torch.no_grad():
            input_tokens = tokenizer(batch['article'], truncation=True, padding='longest', return_tensors="pt")
            target_tokens = tokenizer(batch['summary'], truncation=True, padding='longest', return_tensors="pt")

            input_tokens = {k: v.to(DEVICE) for k, v in input_tokens.items()}
            target_tokens = {k: v.to(DEVICE) for k, v in target_tokens.items()}

            outputs = model(**input_tokens, decoder_input_ids=target_tokens['input_ids'])
            loss = criterion(outputs.logits, target_tokens['input_ids'], target_tokens['attention_mask'])

            val_runnig_loss += loss.item()
            val_runnig_num += len(batch['article'])

            # out_summary = tokenizer.batch_decode(torch.argmax(F.softmax(outputs.logits, dim=-1), dim=-1))
            out_summary = tokenizer.batch_decode(model.module.generate(**input_tokens, min_length=MIN_GEN, do_sample=True, num_beams=3).to('cpu'))
            metric_rouge.add_batch(predictions=out_summary, references=batch['summary'])

        for k, v in input_tokens.items(): del v
        for k, v in target_tokens.items(): del v

        if i > 100:
            break
    
    print('------------------------------')
    print("  running loss val:", val_runnig_loss / val_runnig_num)
    rouge = metric_rouge.compute()
    print(rouge)
    print('------------------------------')

    run["val/rouge1"] = rouge['rouge1'].mid.fmeasure
    run["val/rouge2"] = rouge['rouge2'].mid.fmeasure
    run["val/rougeLsum"] = rouge['rougeLsum'].mid.fmeasure
    run["val/loss"] = val_runnig_loss / val_runnig_num


---
  running loss: 6.196994304656982
---
---
  running loss: 6.640599966049194
---


RuntimeError: ignored

In [ ]:
run.stop()

In [ ]:
rouge['rouge1'].mid.fmeasure

0.14900834094406204

In [ ]:
input_tokens

{'input_ids': tensor([[1027, 1352,    1]]), 'attention_mask': tensor([[1, 1, 1]])}

In [ ]:
import torch
import torch.nn as nn


class Encoder(nn.Module):
    def __init__(self, text_encoder, gat_encoder):
        super().__init__()
        self.text_encoder = encoder
        self.gat_encoder = gat_encoder

    def forward(self, input_ids, attention_mask):
        return self.text_encoder(**kwargs)


class Decoder(nn.Module):
    def __init__(self, base_model):
        super().__init__()
        self.base_model = base_model

    def forward(self, **kwargs):
        return self.base_model(**kwargs)

class Model(nn.Module):
    def __init__(self, base_model):
        super().__init__()
        self.lm_head = Linear(in_features=1024, out_features=96103, bias=False)

    def forward(self, **kwargs):
        return self.base_model(**kwargs)
    

encoder = Encoder(pegasus_model.encoder)
decoder = Decoder(pegasus_model.decoder)

In [ ]:

model.train_model(train_df)

In [ ]:
model.generate(**input_tokens)

KeyError: ignored

In [ ]:
pegasus_model.decoder()

In [ ]:
input_tokens = pegasus_tokenizer(['Some text'], truncation=True, padding='longest', return_tensors="pt")

encoder(**input_tokens).last_hidden_state

tensor([[[ 0.0034,  0.0632,  0.1504,  ...,  0.1808, -0.0844,  0.0342],
         [-0.1237,  0.0421,  0.1656,  ...,  0.1279, -0.3230,  0.0770],
         [ 0.0231, -0.0155,  0.0965,  ...,  0.0193,  0.0016, -0.0204]]],
       grad_fn=<NativeLayerNormBackward>)